<a href="https://colab.research.google.com/github/migostro/laboratorio-de-visao/blob/main/2_2_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install scikit-image==0.19.2

In [ ]:
# utilities
from operator import truediv
import os
import time
import numpy as np
from google.colab import files, drive

# image processing
from scipy import ndimage
from skimage import io, exposure, filters, transform
from skimage import data, measure
from skimage import color
from skimage import morphology
from skimage import segmentation
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from sklearn.metrics import jaccard_score

# visualization
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

# Helper Functions

In [ ]:
def float_to_int(img):
    return (img*255).astype(np.uint8)

In [ ]:
# list_filepaths(): list filepath for every filepath on a root folder
# pre-condition: (root path, empty list)
# post-condition: list with every filepath on root path
def list_filepaths(path, file_extension, filepaths = []):
    if os.path.isdir(path):
        for filename in os.listdir(path):
            filepath = os.path.join(path, filename)
            if os.path.isfile(filepath) and file_extension in filename: filepaths.append(filepath) # Adiciona apenas caminhos que são de arquivos (que no nosso caso são imagens)
            else: list_filepaths(filepath, file_extension, filepaths)

    return filepaths

In [ ]:
def display_single(image):
    fig, ax = plt.subplots()
    ax = plt.imshow(image, cmap=plt.cm.gray)
    plt.show()
    plt.close()

In [ ]:
def gkern(l, sig):
    # creates gaussian kernel with side length `l` and a sigma of `sig` #
    # https://stackoverflow.com/a/43346070 #
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

# Pipeline

In [ ]:
# 0 FILE READING
drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new'
resizedData_path = root + '/originalDataSet_resized'
truth_path = root + "/groundTruth_resized"
segmented_path = root + "/segmented_resized"
bbox_path = root + "/bbox_resized"
bbox_truth_path = root + "/bbox_truth_resized"

resizedData_filepaths = list_filepaths(resizedData_path, 'png', [])
truthData_filepaths = list_filepaths(truth_path, 'png', [])
segmentedData_filepaths = list_filepaths(segmented_path, 'png', [])
print(f'# of images @ {resizedData_path}: {len(resizedData_filepaths)}')
print(f'# of images @ {truth_path}: {len(truthData_filepaths)}')
print(f'# of images @ {segmented_path}: {len(segmentedData_filepaths)}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized: 1080
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth_resized: 160
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/segmented_resized: 1080


### Segmentation

In [ ]:
# 1 SEGMENT
kernel = gkern(3, 1) # default: 3x3 gaussian kernel
classes = np.array(['borrachas', 'cartas', 'celulares', 'conchas', 'copos', 'dados', 'estatuas', 'lapis', 'tubos', 'vasos'])
scores =	{
  'borrachas': [],
  'cartas': [],
  'celulares': [],
  'conchas': [],
  'copos': [],
  'dados': [],
  'estatuas': [],
  'lapis': [],
  'tubos': [],
  'vasos': [],
}

for idx, img_path in enumerate(resizedData_filepaths):
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(resizedData_path, '').replace(filename, '')

    # read
    print(f'[{idx}]', img_path)
    img = io.imread(img_path)

    # segment
    if 'verde' in img_path:
        img = (img*255).astype(np.uint8)
        denoised = filters.rank.median(img, morphology.disk(2))
        gaussian = filters.gaussian(denoised, sigma=.5)
        edges = filters.sobel(gaussian)

        thresholds = filters.threshold_multiotsu(img)

        markers = np.zeros_like(gaussian)
        markers[denoised < thresholds[0]] = 1
        markers[denoised > thresholds[1]] = 2

        segmentation_verde = segmentation.random_walker(edges, markers)
        if np.amax(segmentation_verde) > 1.0: segmentation_verde = segmentation_verde - 1 # fix [1.0, 2.0] range
        segmentation_verde = float_to_int(segmentation_verde) # convert to int
        if segmentation_verde[10,10] == 1:
            segmentation_verde = 1 - segmentation_verde # if background inverted: inverts range
            segmentation_verde = segmentation_verde > 0
        segmentation_verde_holes = morphology.remove_small_holes(segmentation_verde, area_threshold=500)
        segmentation_verde_small = morphology.remove_small_objects(segmentation_verde_holes, min_size=500)

        # write
        if not os.path.isdir(segmented_path + middle_path): os.makedirs(segmented_path + middle_path)
        io.imsave(img_path.replace(resizedData_path, segmented_path).replace('.JPG', '.png'), (segmentation_verde_small*1).astype(np.uint8), check_contrast=False)

    else:
        gaussian = ndimage.convolve(img, kernel, mode='nearest', cval=0.0) # convolves image and 3x3 gaussian kernel

        tresh = filters.threshold_yen(gaussian)
        mask = gaussian < tresh
        if mask[10,10]:
            mask = 1 - mask # if background inverted: inverts range
            mask = mask > 0
        mask_holes = morphology.remove_small_holes(mask, area_threshold=500)
        mask_closing = morphology.binary_closing(mask_holes, footprint=morphology.diamond(3))
        mask_small = morphology.remove_small_objects(mask_closing, min_size=500)

        # write
        if not os.path.isdir(segmented_path + middle_path): os.makedirs(segmented_path + middle_path)
        io.imsave(img_path.replace(resizedData_path, segmented_path).replace('.JPG', '.png'), float_to_int(mask_small), check_contrast=False)

### Jaccard Index

In [ ]:
# 2 JACCARD
classes = np.array(['borrachas', 'cartas', 'celulares', 'conchas', 'copos', 'dados', 'estatuas', 'lapis', 'tubos', 'vasos'])
scores =	{
  'borrachas': [],
  'cartas': [],
  'celulares': [],
  'conchas': [],
  'copos': [],
  'dados': [],
  'estatuas': [],
  'lapis': [],
  'tubos': [],
  'vasos': [],
}
scores_background =	{
  'branco': [],
  'preto': [],
  'verde': [],
}
scores_env =	{
  'exterior': [],
  'interior': [],
}
scores_time =	{
  'dia': [],
  'noite': [],
}

low_contrast = []
not_green = []
not_green_all = []
for idx, img_path in enumerate(segmentedData_filepaths):
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(segmented_path, '').replace(filename, '')

    # read
    hasTruth = False
    # print(f'[{idx}]', img_path)
    # if 'verde' not in img_path: not_green_all.append(img_path.replace(segmented_path, resizedData_path))
    if os.path.isfile(img_path.replace(segmented_path, truth_path).replace('.JPG', '.png')):
        # print(f'[{idx}]', img_path.replace(segmented_path, truth_path).replace('.JPG', '.png'))
        truth = io.imread(img_path.replace(segmented_path, truth_path).replace('.JPG', '.png'))[:,:,0]
        truth = truth > filters.threshold_yen(truth)
        hasTruth = True
    
    if hasTruth:
        img = io.imread(img_path)
        jaccard_idx = jaccard_score(img*1, truth*1, average='micro') # micro p/ 2d image comparison https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html
        curr_class = img_path.split('/')[-3]
        background = img_path.split('/')[-4]
        environment = img_path.split('/')[-5]
        time = img_path.split('/')[-6]
        if curr_class  in scores:            scores[curr_class].append(jaccard_idx)
        if background  in scores_background: scores_background[background].append(jaccard_idx)
        if environment in scores_env:        scores_env[environment].append(jaccard_idx)
        if time        in scores_time:       scores_time[time].append(jaccard_idx)
        # if 'verde' not in img_path: not_green.append(img_path.replace(segmented_path, resizedData_path))
        # print(f'[{idx}]', img_path)
        # if(jaccard_idx < 0.5):
            # print(f'[LOW JACCARD SCORE] {jaccard_idx}')
            # if 'verde' not in img_path: low_contrast.append(img_path.replace(segmented_path, resizedData_path))
            # print(f'[{idx}]', img_path)


### Trials

In [ ]:
kernel = gkern(3, 1) # default: 3x3 gaussian kernel

def invert_mask(mask):
    rr, cc = mask.shape
    d = 50 # distance from image boundaries
    invert_score = 0
    if mask[d,d]: invert_score += 1
    if mask[rr-d, d]: invert_score += 1
    if mask[d, cc-d]: invert_score += 1
    if mask[rr-d, cc-d]: invert_score += 1

    if invert_score > 2:
        mask = 1 - mask # if background inverted: inverts range
        mask = mask > 0 # int to binary
    return mask

for idx, img_path in enumerate(not_green):
    img = io.imread(img_path)
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(resizedData_path, '').replace(filename, '')
    print(f'[{idx}]', img_path)

    # [0] Chooses treshold that yield a mask with the largest contrast and rejects large areas
    gaussian = ndimage.convolve(img, kernel, mode='nearest', cval=0.0) # convolves image and 3x3 gaussian kernel
    yen = filters.threshold_yen(gaussian)
    mask_yen = gaussian < yen
    mask_yen = invert_mask(mask_yen)   # [1] Garantees that background is 0 and object is 1
    otsu = filters.threshold_otsu(gaussian)
    mask_otsu = gaussian < otsu
    mask_otsu = invert_mask(mask_otsu) # [1] Garantees that background is 0 and object is 1

    background_treshold = 0.85*(img.shape[0]*img.shape[1]/2)
    area_yen = np.sum(mask_yen)
    area_otsu = np.sum(mask_otsu)
    if (area_yen > background_treshold) or (area_otsu > background_treshold): # rejects if area of mask is more than 85% of half of the area of the image
        if (area_yen > background_treshold): mask = mask_otsu
        else: mask = mask_yen
    else:
        if(mask_yen.std() > mask_otsu.std()): mask = mask_yen # RMS Contrast https://en.wikipedia.org/wiki/Contrast_(vision)#RMS_contrast
        else: mask = mask_otsu

    # [2] Morphological operations
    mask = morphology.remove_small_holes(mask, area_threshold=500)          # holes
    mask = morphology.binary_closing(mask, footprint=morphology.diamond(3)) # closing gaps
    mask = morphology.remove_small_objects(mask, min_size=500)              # small objects
    mask = ndimage.binary_fill_holes(mask, np.ones((3,) * mask.ndim))       # larger holes

    # [3] Isolates object and extracts bounding box
    label_img = measure.label(mask)
    regions = measure.regionprops(label_img)

    if(len(regions) > 1):
        # isolation
        center = (int(mask.shape[0]/2), int(mask.shape[1]/2)) # image center
        most_centered_region = min(regions, key=lambda region: (np.linalg.norm(np.subtract(region.centroid, center))/region.area) ) # minimizes center distance and maximizes object area
        mask_isolated = np.zeros(mask.shape)
        mask_isolated[most_centered_region.slice] = mask[most_centered_region.slice] # object isolation

        # write mask
        if not os.path.isdir(segmented_path + middle_path): os.makedirs(segmented_path + middle_path)
        io.imsave(img_path.replace(resizedData_path, segmented_path), float_to_int(mask), check_contrast=False)

        # write bounding box
        if not os.path.isdir(bbox_path + middle_path): os.makedirs(bbox_path + middle_path)
        io.imsave(img_path.replace(resizedData_path, bbox_path), most_centered_region.image)
    else:
        # write mask
        if not os.path.isdir(segmented_path + middle_path): os.makedirs(segmented_path + middle_path)
        io.imsave(img_path.replace(resizedData_path, segmented_path), float_to_int(mask), check_contrast=False)

        # write bounding box
        if not os.path.isdir(bbox_path + middle_path): os.makedirs(bbox_path + middle_path)
        io.imsave(img_path.replace(resizedData_path, bbox_path), regions[0].image)

In [ ]:
# get average jaccard scores
print('[1.1] Average Jaccard Scores by Class')
for key, value in scores.items():
    if value: print(key, np.average(value), np.median(value), len(value))
print('\n[1.2] Average Jaccard Scores by Background')
for key, value in scores_background.items():
    if value: print(key, np.average(value), np.median(value), len(value))
print('\n[1.3] Average Jaccard Scores by Environment')
for key, value in scores_env.items():
    if value: print(key, np.average(value), np.median(value), len(value))
print('\n[1.4] Average Jaccard Scores by Time')
for key, value in scores_time.items():
    if value: print(key, np.average(value), np.median(value), len(value))

[1.1] Average Jaccard Scores by Class
borrachas 0.5028683849566293 0.5372073393809726 16
cartas 0.8879609751064526 0.9654714741382896 16
celulares 0.8587511738979308 0.9382834412314495 16
conchas 0.7679160444890498 0.9453178330991464 16
copos 0.583611312522249 0.5869248759873509 16
dados 0.5013776489790986 0.6070948436590726 16
estatuas 0.5786711861055509 0.668527715852139 16
lapis 0.64614545721525 0.6081752987014435 16
tubos 0.714402874720824 0.912002903974801 16
vasos 0.7008440854595189 0.7404787566656339 16

[1.2] Average Jaccard Scores by Background
branco 0.7489363669904323 0.7360050678491648 63
preto 0.8451549150921231 0.9169614364258526 53
verde 0.36146783352184303 0.40740540305560063 44

[1.3] Average Jaccard Scores by Environment
exterior 0.6985966532794831 0.7318728878875912 79
interior 0.6505142060019964 0.7166952444273367 81

[1.4] Average Jaccard Scores by Time
dia 0.6371364224692099 0.6706609276715987 120
noite 0.7856103899733917 0.9073871602337155 40
